In [1]:
#importing required libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

import torch 
import numpy as np 
import torchvision.transforms as transforms 
import torchvision 
import torch.nn.functional as F

In [7]:
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])



dataset_train = CIFAR10(root='../data', train=True, 
            download=True, transform=transform)
dataset_test = CIFAR10(root='../data', train=False, 
             download=True, transform=transform)
train_loader = DataLoader(dataset_train, batch_size=32, 
                        shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=32, 
                       shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
class baseBlock(torch.nn.Module):
    expansion = 1
    def __init__(self,input_planes,planes,stride=1,dim_change=None):
        super(baseBlock,self).__init__()
        #declare convolutional layers with batch norms
        self.conv1 = torch.nn.Conv2d(input_planes,planes,stride=stride,kernel_size=3,padding=1)
        self.bn   = torch.nn.BatchNorm2d(planes)
        self.conv2 = torch.nn.Conv2d(planes,planes,stride=1,kernel_size=3,padding=1)
        self.dim_change = dim_change
        
    def forward(self,x):
        #Save the residue
        res = x
        output = F.relu(self.bn(self.conv1(x)))
        output = self.bn(self.conv2(output))

        if self.dim_change is not None:
            res = self.dim_change(res)
        
        output = output + res
        output = F.relu(output)

        return output


In [ ]:
class ResNet18(torch.nn.Module):
    def __init__(self,block,num_layers,classes=10):
        super(ResNet18,self).__init__()
        #according to research paper:
        self.input_planes = 64
        self.conv1 = torch.nn.Conv2d(3,64,kernel_size=7,stride=2, padding=5)
        self.bn1   = torch.nn.BatchNorm2d(64)
        self.max1  = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = torch.nn.Sequential(
                            block(self.input_planes, planes),
                            block(planes*expansion, planes),
                            
        self.layer1 = torch.nn.Sequential(
                            torch.nn.Conv2d(self.input_planes,planes*block.expansion, kernel_size=1, stride=stride),
                            torch.nn.BatchNorm2d(planes*block.expansion)
                        )
        
        self.layer1 = self._layer(block,64,2,stride=1)
        self.layer2 = self._layer(block,128,2,stride=2)
        self.layer3 = self._layer(block,256,2,stride=2)
        self.layer4 = self._layer(block,512,2,stride=2)
        self.averagePool = torch.nn.AvgPool2d(kernel_size=4,stride=1)
        self.fc    =  torch.nn.Linear(512*block.expansion,classes)
    
    def _layer(self,block,planes,num_layers,stride=1):
        dim_change = None
        if stride!=1 or planes != self.input_planes*block.expansion:
            dim_change = torch.nn.Sequential(torch.nn.Conv2d(self.input_planes,planes*block.expansion,kernel_size=1,stride=stride),
                                             torch.nn.BatchNorm2d(planes*block.expansion))
        netLayers =[]
        netLayers.append(block(self.input_planes,planes,stride=stride,dim_change=dim_change))
        self.input_planes = planes * block.expansion
        for i in range(1,num_layers):
            netLayers.append(block(self.input_planes,planes))
            self.input_planes = planes * block.expansion
        
        return torch.nn.Sequential(*netLayers)

    def forward(self,x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.max1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = F.avg_pool2d(x,4)
        x = x.view(x.size(0),-1)
        x = self.fc(x)

        return x


## Resnet 18

In [3]:
class bottleNeck(torch.nn.Module):
    expansion = 4
    def __init__(self,input_planes,planes,stride=1,dim_change=None):
        super(bottleNeck,self).__init__()

        self.conv1 = torch.nn.Conv2d(input_planes,planes,kernel_size=1,stride=1)
        self.bn = torch.nn.BatchNorm2d(planes)
        self.conv2 = torch.nn.Conv2d(planes,planes,kernel_size=3,stride=stride,padding=1)
        self.conv3 = torch.nn.Conv2d(planes,planes*self.expansion,kernel_size=1)
        self.bn2 = torch.nn.BatchNorm2d(planes*self.expansion)
        self.dim_change = dim_change
    
    def forward(self,x):
        res = x
        
        output = F.relu(self.bn(self.conv1(x)))
        output = F.relu(self.bn(self.conv2(output)))
        output = self.bn2(self.conv3(output))

        if self.dim_change is not None:
            res = self.dim_change(res)
        
        output = output + res
        output = F.relu(output)
        return output